In [1]:
import os
import json
import numpy as np
import robotic as ry
from tqdm import tqdm
from lookup import features, contraint_types
from komo_generators import generate_random_komo, generate_random_pick, generate_random_pivot
from random_kitchen_generator import generate_random_kitchen, load_single_from_type

In [ ]:
C = generate_random_kitchen(seed=89032, root_path="/home/denis/rai-robotModels/robocasa/")
frame_names = C.getFrameNames()
features["qItself"][1] = len(C.getJointState())
object_frames = []
with open("random_objs.g", 'r', encoding='utf-8') as file:
    for line in file:
        words = line.strip().split()
        if words:  # Ensure the line is not empty
            object_frames.append(words[0])
C = load_single_from_type(C, pos=[.5, 1., 1.2], root_path=f"/home/denis/rai-robotModels/robocasa/rai_jointed/fixtures/sinks")

import time
np.random.seed(int(time.time()))

## Generate KOMO texts

In [3]:
# Special Features (for pure random KOMOs)
lucky_frames = ["r_gripper", "r_palm"]
lucky_features = ["positionDiff", "position", "vectorX", "vectorY", "vectorZ", "quaternion", "negDistance"]

# Special Features (for random manipulation KOMOs)
end_effector_frames = ["r_gripper"]
mobile_base_frames = ["omnibase"]
# object_frames = ["kebabs_8", "can_6", "pizza_cutter_2", "bell_pepper_0", "kebabs_13"]  # Frames that are not attached to anything that can be manipulated with basic pick, place, push, etc.
joint_frames = ["spout_joint", "handle_joint"]
jointed_frames = ["spout_visual0", "handle_visual0"]  # Frames that have a parent / grandparent / great-grandparent / etc. that is a joint (Not including robot)
world_attach_frames = ["floor"]  # Like table in manipulation.py, some sort of world frame that tells us the world rotation and for us to place stuff after a mode switch.

In [4]:
# komo_generator = lambda: generate_random_komo(frame_names, features, lucky_frames, lucky_features, prob_mode_switch=0)
# komo_generator = lambda: generate_random_pick(end_effector_frames,
#                                               object_frames,
#                                               world_attach_frames,
#                                               frame_names,
#                                               drop_prob=0,
#                                               specific_orientation_prob=1,
#                                               specific_place_prob=0,
#                                               place_prob=1)
# komo_generator = lambda: generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=8)
def komo_generator():
    if np.random.random() < .65:
        text = generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=8)
    else:
        text = generate_random_pick(end_effector_frames,
                                    object_frames,
                                    world_attach_frames,
                                    frame_names,
                                    drop_prob=0,
                                    specific_orientation_prob=1,
                                    specific_place_prob=0,
                                    place_prob=1)
    return text

In [ ]:
for i in range(10):
    komo_text = komo_generator()
    print(komo_text)

In [ ]:
komo_samples = 10_000

outfolder = "./komo_samples"
text_outfolder = os.path.join(outfolder, "texts")
if not os.path.exists(text_outfolder):
    os.makedirs(text_outfolder)
else:
    for file in os.listdir(text_outfolder):
        file_path = os.path.join(text_outfolder, file)
        os.remove(file_path)

for i in tqdm(range(komo_samples)):
    komo_text = komo_generator()
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "w") as file:
        file.write(komo_text)

## Classify KOMO texts

In [ ]:
total_komos = 0
total_feasible_komos = 0
total_error_komos = 0

feasible_komos = []
error_komos = []

for i in tqdm(range(komo_samples)):
    
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()

    # print(f"Processing file {filename}...")
    total_komos += 1
    
    # try:
    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    # except:
    #     print(komo_text)
    #     exit()
    #     total_error_komos += 1
    #     error_komos.append(i)
    #     continue
    
    if ret.feasible:
        total_feasible_komos += 1
        feasible_komos.append(i)
    
    if (i+1)%(komo_samples//10) == 0:
        print(f"Total feasible komos: {total_feasible_komos}")
        print(f"Total error komos: {total_error_komos}")
        print(f"Total komos: {total_komos}")
        print(f"{(total_feasible_komos/total_komos)*100:.2f}% feasible")

In [ ]:
result = f"""Total feasible komos - Total error komos - Total komos: 
{total_feasible_komos} - {total_error_komos} - {total_komos}. 
({(total_feasible_komos/total_komos)*100:.2f}% feasible)"""
print(result)
print(feasible_komos)
results_file_path = os.path.join(outfolder, "feasibles.json")
results_dict = {
    "feasible_komos": feasible_komos,
    "error_komos": error_komos,
}
json.dump(results_dict, open(results_file_path, 'w'))

## Visualize feasible KOMOs

In [ ]:
# outfolder = "komo_samples_pick"
# text_outfolder = os.path.join(outfolder, "text")
# feasible_komos = json.load(open(os.path.join(outfolder, "feasibles.json"), 'r'))["feasible_komos"]

feasibe_samples_count = 10
if len(feasible_komos) < feasibe_samples_count:
    feasibe_samples = feasible_komos.copy()
else:
    feasibe_samples = np.random.choice(feasible_komos, feasibe_samples_count, replace=False)
    print(feasibe_samples)
    feasibe_samples = [6054, 3687, 7429, 6099, 95, 1401, 305, 424, 8623, 5929, 5736]

for i in feasibe_samples:

    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()
    print(f"__________ SAMPLE {i} __________")
    print(komo_text)

    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()

    komo.view(True)
    komo.view_play(True)